In [333]:
"""
Build Broadlink RM-series RF packets (433 MHz).

Usage
-----
from broadlink_rf_builder import build_rf_packet

# fan remote — needs an extra 92 805 µs 'tiny gap':
packet = build_rf_packet(
    "10101101010010001010100",
    initial_gap_us=92_805       # <-- here
)
device.send_data(packet)
"""

from __future__ import annotations
from typing import Iterable, List, Sequence, Union

TICK_US = 32.84                      # Broadlink time-unit

# -------------- handset-style defaults (override per call) ----------
HEADER_RF_433    = b"\xB2"
MARK0_US, SPACE0_US = 394, 755       # 0 bit  (12 u, 23 u)
MARK1_US, SPACE1_US = 755, 394       # 1 bit  (23 u, 12 u)
LEAD_IN_US           = 1_336_916
INITIAL_GAP_US       = 92_805             # <-- 92_805 for your fan
FIRST_PREAMBLE_US = (
    MARK1_US, SPACE1_US,  # 0
    MARK1_US              # 1    → pattern “110” (sync, 1 bit)
)
PREAMBLE_US          = (
    MARK1_US, SPACE1_US,  # 1
    MARK0_US, SPACE0_US,  # 0
    MARK0_US, SPACE0_US,  # 0  -> pattern “100” (sync)
)
INTER_GAP_US         = (11_822, SPACE1_US)  # 50 ms between repeats
TRAILER_US           = 49_260
REPEATS              = 5
# --------------------------------------------------------------------


def _ceil_tick(us: float) -> int:
    return int(round(us / TICK_US))


def _normalise_to_bits(src: Union[str, bytes, bytearray, Sequence[int]]) -> str:
    if isinstance(src, str):
        s = src.strip().lower().replace("0x", "")
        return s if set(s) <= {"0", "1"} else bin(int(s, 16))[2:]
    if isinstance(src, (bytes, bytearray)):
        iterable: Iterable[int] = src
    else:
        iterable = src
    return "".join(f"{b:08b}" for b in iterable)


def _bits_to_pulses(bits: str,
                    mark0: int, space0: int,
                    mark1: int, space1: int) -> List[int]:
    out: List[int] = []
    for bit in bits:
        out += (mark0, space0) if bit == "0" else (mark1, space1)
    return out


def _encode(pulses_us: Sequence[int], repeat: int) -> bytearray:
    buf = bytearray(b"\x26\x00\x00\x00")           # IR placeholder
    for us in pulses_us:
        t = _ceil_tick(us)
        if t >= 256:
            buf += b"\x00" + t.to_bytes(2, "big")
        else:
            buf.append(t)
    buf[2:4] = (len(buf) - 4).to_bytes(2, "little")
    buf[0] = 0xB2                      # flip to RF
    buf[1] = repeat.to_bytes(1, "little")[0]  # repeat count
    return buf

def build_rf_packet(
    payload: Union[str, bytes, bytearray, Sequence[int]],
    *,
    pair: bool = False,  # whether to use pair inter-gap
    repeats: int = REPEATS,
    lead_in_us: int = LEAD_IN_US,
    initial_gap_us: int = INITIAL_GAP_US,         # <─ tiny gap lives here
    inter_gap_us: Sequence[int] = INTER_GAP_US,
    trailer_us: int = TRAILER_US,
    mark0_us: int = MARK0_US, space0_us: int = SPACE0_US,
    mark1_us: int = MARK1_US, space1_us: int = SPACE1_US,
    first_preamble_us: Sequence[int] = FIRST_PREAMBLE_US,
    preamble_us: Sequence[int] = PREAMBLE_US,
    
) -> bytes:
    """Return a Broadlink RF packet ready for ``device.send_data()``."""
    bits   = _normalise_to_bits(payload)
    frame  = _bits_to_pulses(bits,
                                mark0_us, space0_us,
                                mark1_us, space1_us)

    pulses = [lead_in_us]
    if initial_gap_us:
        pulses.append(initial_gap_us)
    pulses +=  list(first_preamble_us) + frame

    for i in range(repeats - 1):
        pulses += list(inter_gap_us) + list(preamble_us) + frame
    if not pair:
        pulses.append(trailer_us)
    print("My pulses:", "".join(str(b) + ', ' for b in pulses))

    return bytes(_encode(pulses, 0xC9 if pair else 0xEF))  # 0xC1 for pairing, 0xC0 for normal
def build_pair_packet(payload: Union[str, bytes, bytearray, Sequence[int]],
):
    """Build a pairing packet with the default parameters."""
    return build_rf_packet(
        payload,
        pair=True,                   # pairing mode
        repeats=10,                  # number of repeats
    )
# ---------------- example usage ------------------------------------
LIGHT_ON_BITSTRING = "10101101010010001010100"
DIM_UP_BITSTRING = "10101101010010000010010"
TEST_BITSTRING = "10101101010011111000000"
packet = build_rf_packet(LIGHT_ON_BITSTRING, repeats=3, trailer_us=394)         # almost no tail)
pairing_packet = build_rf_packet(TEST_BITSTRING, pair=True, repeats=10)
print("My command:", "".join('0x'+format(b, '02x')+', ' for b in packet))


print(f"Broadlink packet ({len(packet)} bytes):\n{packet.hex()}")
# device.send_data(packet)   # uncomment when you have the device instance

My pulses: 1336916, 92805, 755, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 394, 755, 755, 394, 394, 755, 394, 755, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 394, 755, 11822, 394, 755, 394, 394, 755, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 394, 755, 755, 394, 394, 755, 394, 755, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 394, 755, 11822, 394, 755, 394, 394, 755, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 394, 755, 755, 394, 394, 755, 394, 755, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 394, 755, 394, 
My pulses: 1336916, 92805, 755, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 755, 394, 394, 755, 755, 394, 394, 755, 755, 394, 394, 755, 394, 755, 755, 394, 755, 3

In [335]:
import broadlink
device = broadlink.hello("192.168.1.89")
device.auth()
device

broadlink.remote.rm4pro(('192.168.1.89', 80), mac=b'\xe8pr\x08l\xf1', devtype=21003, timeout=10, name='Rm4 pro', model='RM4 pro', manufacturer='Broadlink', is_locked=False)

In [336]:
# for i in range(25): 
device.send_data(packet)   # uncomment when you have the device instance

In [300]:
device.send_data(pairing_packet)  # send the first packet

In [157]:
# integer to byte
num = 9
print(f"Integer {num} as byte: {num.to_bytes(1, 'little')}")

Integer 9 as byte: b'\t'
